In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

from torch import nn
import torch
import torch.nn.functional as F
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gc

: 

In [150]:
import pytorch_lightning as pl
python -m ipykernel install --user --name recsys --display-name recsys

ImportError: cannot import name 'Self' from 'typing_extensions' (/opt/conda/lib/python3.8/site-packages/typing_extensions.py)

In [26]:
class Config:
    device = torch.device("cuda")
    MAX_SEQ = 100
    EMBED_DIMS = 512
    ENC_HEADS = DEC_HEADS = 8
    NUM_ENCODER = NUM_DECODER = 4
    BATCH_SIZE = 32
    TRAIN_FILE = "/opt/ml/input/data/train_data.csv"
    TEST_FILE = "/opt/ml/input/data/test_data.csv"
    TOTAL_EXE = 13523
    TOTAL_CAT = 10000
    AUGMENTATION = 5


# 1.데이터 불러오기

기존 SAINT : train의 Column 종류

row_id, timestamp, user_id, content_id, content_type_id,

task_container_id, user_answer, answered_correctly, prior_question_elapsed_time, prior_question_had_explanation

우리 DKT

userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag

공통적으로 User id, timestamp 있고

콘텐츠적으로 나누면 우리 dkt 는 3개정도 분류될거 같고,

saint+이기 때문에 time 에 관해서 더 프로세싱하면 될거 같다.(elapsed, Lagged time 은 데이터상으로 구별하기 힘들다.)


In [13]:
class DKTDataset(Dataset):
    def __init__(self, samples, max_seq):
        super().__init__()
        self.samples = samples
        self.max_seq = max_seq
        self.data = []
        for id in self.samples.index:
            exe_ids, answers, ela_time, categories = self.samples[id]
            if len(exe_ids) > max_seq:
                for l in range((len(exe_ids)+max_seq-1)//max_seq):
                    self.data.append(
                        (exe_ids[l:l+max_seq], answers[l:l+max_seq], ela_time[l:l+max_seq], categories[l:l+max_seq]))
            elif len(exe_ids) < self.max_seq and len(exe_ids) > 50:
                self.data.append((exe_ids, answers, ela_time, categories))
            else:
                continue

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question_ids, answers, ela_time, exe_category = self.data[idx]
        seq_len = len(question_ids)

        exe_ids = np.zeros(self.max_seq, dtype=int)
        ans = np.zeros(self.max_seq, dtype=int)
        elapsed_time = np.zeros(self.max_seq, dtype=int)
        exe_cat = np.zeros(self.max_seq, dtype=int)
        if seq_len < self.max_seq:
            exe_ids[-seq_len:] = question_ids
            ans[-seq_len:] = answers
            elapsed_time[-seq_len:] = ela_time
            exe_cat[-seq_len:] = exe_category
        else:
            exe_ids[:] = question_ids[-self.max_seq:]
            ans[:] = answers[-self.max_seq:]
            elapsed_time[:] = ela_time[-self.max_seq:]
            exe_cat[:] = exe_category[-self.max_seq:]

        input_rtime = np.zeros(self.max_seq, dtype=int)
        input_rtime = np.insert(elapsed_time, 0, 0)
        input_rtime = np.delete(input_rtime, -1)

        input = {"input_ids": exe_ids, "input_rtime": input_rtime.astype(
            np.int), "input_cat": exe_cat}
        return input, ans


전처리


'timestamp': 'int64',  -> Timestamp

'user_id': 'int32',  -> userID

'content_id': 'int16' -> assessmentItemID

'answered_correctly': 'int8',   -> answerCode



"content_type_id": "int8", -> 버림



"prior_question_elapsed_time": "float32", -> Timestamp 변형

"task_container_id": "int16" -> 누적 개수 (새로 생성 필요)



In [120]:
print("loading csv.....")
train_df = pd.read_csv(Config.TRAIN_FILE)
test_df = pd.read_csv(Config.TEST_FILE)
print("shape of (train) dataframe :", train_df.shape)

# 기본적인 데이터 전처리 부분.
# column : userID assessmentItemID  testId answerCode Timestamp KnowledgeTag grade

train_df['grade'] = (train_df['assessmentItemID'].str[2]).astype('int')
test_df['grade'] = (test_df['assessmentItemID'].str[2]).astype('int')
train_df['testId'] = (train_df['assessmentItemID'].str[2] + train_df['assessmentItemID'].str[4:7])
test_df['testId'] = test_df['assessmentItemID'].str[2] + test_df['assessmentItemID'].str[4:7]
train_df["Timestamp"] = pd.to_datetime(train_df["Timestamp"])
test_df["Timestamp"] = pd.to_datetime(test_df["Timestamp"])

ItemID2idx = {v:k for k,v in enumerate(train_df['assessmentItemID'].unique())} # 9453개
testId2idx = {v:k for k,v in enumerate(train_df['testId'].unique())} # 1536개
Tag2idx = {v:k for k,v in enumerate(train_df['KnowledgeTag'].unique())} # 911개

# args.n_item = train_df['assessmentItemID'].nunique()
# args.n_test = train_df['testId'].nunique()
# args.n_tag = train_df['KnowledgeTag'].nunique()
# args.n_grade = train_df['grade'].nunique()

train_df['assessmentItemID'] = train_df['assessmentItemID'].map(ItemID2idx)
train_df['testId'] = train_df['testId'].map(testId2idx)
train_df['KnowledgeTag'] = train_df['KnowledgeTag'].map(Tag2idx)

test_df['assessmentItemID'] = test_df['assessmentItemID'].map(ItemID2idx)
test_df['testId'] = test_df['testId'].map(testId2idx)
test_df['KnowledgeTag'] = test_df['KnowledgeTag'].map(Tag2idx)



#필요 Column 생성 - 'content_type_id'
train_df['content_type_id'] = 0
test_df['content_type_id'] = 0

#필요 Column 생성 - 'prior_question_elapsed_time'
train_df['prior_question_elapsed_time'] = 0
test_df['prior_question_elapsed_time'] = 0

for i in range(len(train_df)-1):
    if train_df.at[i,'userID'] == train_df.at[(i+1),'userID']:
        train_df.at[i,'prior_question_elapsed_time'] = (train_df.at[i+1,'Timestamp']-train_df.at[i,'Timestamp']).total_seconds()
    if i %100000 == 0:
        print(f'train elapsetime process  {i}/{len(train_df)} done ')

for i in range(len(test_df)-1):
    if test_df.at[i,'userID'] == test_df.at[(i+1),'userID']:
        test_df.at[i,'prior_question_elapsed_time'] = (test_df.at[i+1,'Timestamp']-test_df.at[i,'Timestamp']).total_seconds()
    if i %100000 == 0:
        print(f'test elpasetime process  {i}/{len(train_df)} done ')

# 필요 Column 생성 - 'task_container_id'
train_df['task_container_id'] = 0
test_df['task_container_id'] = 0

for i in range(1,len(train_df)):
    if train_df.at[i,'userID'] == train_df.at[i-1,'userID']:
        train_df.at[i,'task_container_id'] = train_df.at[i-1,'task_container_id'] + 1
    if i %100000 == 0:
        print(f'train task process  {i}/{len(train_df)} done ')

for i in range(1,len(test_df)):
    if test_df.at[i,'userID'] == test_df.at[i-1,'userID']:
        test_df.at[i,'task_container_id'] = test_df.at[i-1,'task_container_id'] + 1
    if i %100000 == 0:
        print(f'test task process  {i}/{len(train_df)} done ')

# SAINT에 필요한 것만 남김
train_df = train_df.drop(['testId','KnowledgeTag','grade'], axis=1)
test_df = test_df.drop(['testId','KnowledgeTag','grade'], axis=1)

# change to SAINT Input
train_df  = train_df.rename(columns = {'userID': 'user_id', 'Timestamp':'timestamp', 'assessmentItemID' : 'content_id',
                           'answerCode' : 'answered_correctly'  })
test_df  = test_df.rename(columns = {'userID': 'user_id', 'Timestamp':'timestamp', 'assessmentItemID' : 'content_id',
                           'answerCode' : 'answered_correctly'  })


loading csv.....
shape of (train) dataframe : (2266586, 6)
train elapsetime process  0/2266586 done 
train elapsetime process  100000/2266586 done 
train elapsetime process  200000/2266586 done 
train elapsetime process  300000/2266586 done 
train elapsetime process  400000/2266586 done 
train elapsetime process  500000/2266586 done 
train elapsetime process  600000/2266586 done 
train elapsetime process  700000/2266586 done 
train elapsetime process  800000/2266586 done 
train elapsetime process  900000/2266586 done 
train elapsetime process  1000000/2266586 done 
train elapsetime process  1100000/2266586 done 
train elapsetime process  1200000/2266586 done 
train elapsetime process  1300000/2266586 done 
train elapsetime process  1400000/2266586 done 
train elapsetime process  1500000/2266586 done 
train elapsetime process  1600000/2266586 done 
train elapsetime process  1700000/2266586 done 
train elapsetime process  1800000/2266586 done 
train elapsetime process  1900000/2266586 do

In [139]:
#data augmentation
train_origin = train_df.copy()
n= 1
print(f'======origin length      : {len(train_df)}======')
for i in range(Config.AUGMENTATION):
    print(f'START {n}th AUGMENTATION')
    tem = train_origin.drop_duplicates(subset = ["user_id"],keep = "last")
    train_origin = train_origin.drop(index=tem.index)
    train_origin['user_id'] += train_origin['user_id'].nunique()
    train_df = pd.concat([train_df, train_origin], axis = 0)
    print(f'END   {n}th AUGMENTATION')
    n += 1
print(f'======after augmentation : {len(train_df)}======')

======origin length      : 2266586======
START 1th AUGMENTATION
END   1th AUGMENTATION
START 2th AUGMENTATION
END   2th AUGMENTATION
START 3th AUGMENTATION
END   3th AUGMENTATION
START 4th AUGMENTATION
END   4th AUGMENTATION
START 5th AUGMENTATION
END   5th AUGMENTATION
======after augmentation : 13499046======


In [142]:
train_df = train_df[train_df.content_type_id == 0]
train_df.prior_question_elapsed_time.fillna(0, inplace=True)
train_df.prior_question_elapsed_time /= 1000
# train_df.prior_question_elapsed_time.clip(lower=0,upper=300,inplace=True)
train_df.prior_question_elapsed_time = train_df.prior_question_elapsed_time.astype(
    np.int)

train_df = train_df.sort_values(
    ["timestamp"], ascending=True).reset_index(drop=True)
n_skills = train_df.content_id.nunique()
print("no. of skills :", n_skills)
print("shape after exlusion:", train_df.shape)

# grouping based on user_id to get the data supplu
print("Grouping users...")
group = train_df[["user_id", "content_id", "answered_correctly", "prior_question_elapsed_time", "task_container_id"]]\
    .groupby("user_id")\
    .apply(lambda r: (r.content_id.values, r.answered_correctly.values,
                        r.prior_question_elapsed_time.values, r.task_container_id.values))
# del train_df
# gc.collect()
print("splitting")
train, val = train_test_split(group, test_size=0.2)
print("train size: ", train.shape, "validation size: ", val.shape)




no. of skills : 9454
shape after exlusion: (13499046, 7)
Grouping users...
splitting
train size:  (29754,) validation size:  (7439,)


In [143]:

train_dataset = DKTDataset(train, max_seq=Config.MAX_SEQ)
val_dataset = DKTDataset(val, max_seq=Config.MAX_SEQ)

train_loader = DataLoader(train_dataset,
                            batch_size=Config.BATCH_SIZE,
                            num_workers=8,
                            shuffle=True)
val_loader = DataLoader(val_dataset,
                        batch_size=Config.BATCH_SIZE,
                        num_workers=8,
                        shuffle=False)
# del train_dataset, val_dataset
# gc.collect()

train_loader, val_loader = get_dataloaders()

# 2. 모델

## 2.1 멀티헤드어텐션

In [144]:
class StackedNMultiHeadAttention(nn.Module):
    def __init__(self, n_stacks, n_dims, n_heads, seq_len, n_multihead=1, dropout=0.0):
        super(StackedNMultiHeadAttention, self).__init__()
        self.n_stacks = n_stacks
        self.n_multihead = n_multihead
        self.n_dims = n_dims
        self.norm_layers = nn.LayerNorm(n_dims)
        # n_stacks has n_multiheads each
        self.multihead_layers = nn.ModuleList(n_stacks*[nn.ModuleList(n_multihead*[nn.MultiheadAttention(embed_dim=n_dims,
                                                                                                         num_heads=n_heads,
                                                                                                         dropout=dropout), ]), ])
        self.ffn = nn.ModuleList(n_stacks*[FFN(n_dims)])
        self.mask = torch.triu(torch.ones(seq_len, seq_len),
                               diagonal=1).to(dtype=torch.bool)

    def forward(self, input_q, input_k, input_v, encoder_output=None, break_layer=None):
        for stack in range(self.n_stacks):
            for multihead in range(self.n_multihead):
                norm_q = self.norm_layers(input_q)
                norm_k = self.norm_layers(input_k)
                norm_v = self.norm_layers(input_v)
                heads_output, _ = self.multihead_layers[stack][multihead](query=norm_q.permute(1, 0, 2),
                                                                          key=norm_k.permute(
                                                                              1, 0, 2),
                                                                          value=norm_v.permute(
                                                                              1, 0, 2),
                                                                          attn_mask=self.mask.to(Config.device))
                heads_output = heads_output.permute(1, 0, 2)
                #assert encoder_output != None and break_layer is not None
                if encoder_output != None and multihead == break_layer:
                    assert break_layer <= multihead, " break layer should be less than multihead layers and postive integer"
                    input_k = input_v = encoder_output
                    input_q = input_q + heads_output
                else:
                    input_q = input_q + heads_output
                    input_k = input_k + heads_output
                    input_v = input_v + heads_output
            last_norm = self.norm_layers(heads_output)
            ffn_output = self.ffn[stack](last_norm)
            ffn_output = ffn_output + heads_output
        # after loops = input_q = input_k = input_v
        return ffn_output

## 2.2 임베딩

In [145]:
class EncoderEmbedding(nn.Module):
    def __init__(self, n_exercises, n_categories, n_dims, seq_len):
        super(EncoderEmbedding, self).__init__()
        self.n_dims = n_dims
        self.seq_len = seq_len
        self.exercise_embed = nn.Embedding(n_exercises, n_dims)
        self.category_embed = nn.Embedding(n_categories, n_dims)
        self.position_embed = nn.Embedding(seq_len, n_dims)

    def forward(self, exercises, categories):
        e = self.exercise_embed(exercises)
        c = self.category_embed(categories)
        seq = torch.arange(self.seq_len, device=Config.device).unsqueeze(0)
        p = self.position_embed(seq)
        return p + c + e

In [146]:
class DecoderEmbedding(nn.Module):
    def __init__(self, n_responses, n_dims, seq_len):
        super(DecoderEmbedding, self).__init__()
        self.n_dims = n_dims
        self.seq_len = seq_len
        self.response_embed = nn.Embedding(n_responses, n_dims)
        self.time_embed = nn.Linear(1, n_dims, bias=False)
        self.position_embed = nn.Embedding(seq_len, n_dims)

    def forward(self, responses):
        e = self.response_embed(responses)
        seq = torch.arange(self.seq_len, device=Config.device).unsqueeze(0)
        p = self.position_embed(seq)
        return p + e

## 2.3 메인

In [ ]:
class PlusSAINTModule(pl.LightningModule):
    def __init__(self):
        # n_encoder,n_detotal_responses,seq_len,max_time=300+1
        super(PlusSAINTModule, self).__init__()
        self.loss = nn.BCEWithLogitsLoss()
        self.encoder_layer = StackedNMultiHeadAttention(n_stacks=Config.NUM_DECODER,
                                                        n_dims=Config.EMBED_DIMS,
                                                        n_heads=Config.DEC_HEADS,
                                                        seq_len=Config.MAX_SEQ,
                                                        n_multihead=1, dropout=0.0)
        self.decoder_layer = StackedNMultiHeadAttention(n_stacks=Config.NUM_ENCODER,
                                                        n_dims=Config.EMBED_DIMS,
                                                        n_heads=Config.ENC_HEADS,
                                                        seq_len=Config.MAX_SEQ,
                                                        n_multihead=2, dropout=0.0)
        self.encoder_embedding = EncoderEmbedding(n_exercises=Config.TOTAL_EXE,
                                                  n_categories=Config.TOTAL_CAT,
                                                  n_dims=Config.EMBED_DIMS, seq_len=Config.MAX_SEQ)
        self.decoder_embedding = DecoderEmbedding(
            n_responses=3, n_dims=Config.EMBED_DIMS, seq_len=Config.MAX_SEQ)
        self.elapsed_time = nn.Linear(1, Config.EMBED_DIMS)
        self.fc = nn.Linear(Config.EMBED_DIMS, 1)

    def forward(self, x, y):
        enc = self.encoder_embedding(
            exercises=x["input_ids"], categories=x['input_cat'])
        dec = self.decoder_embedding(responses=y)
        elapsed_time = x["input_rtime"].unsqueeze(-1).float()
        ela_time = self.elapsed_time(elapsed_time)
        dec = dec + ela_time
        # this encoder
        encoder_output = self.encoder_layer(input_k=enc,
                                            input_q=enc,
                                            input_v=enc)
        #this is decoder
        decoder_output = self.decoder_layer(input_k=dec,
                                            input_q=dec,
                                            input_v=dec,
                                            encoder_output=encoder_output,
                                            break_layer=1)
        # fully connected layer
        out = self.fc(decoder_output)
        return out.squeeze()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def training_step(self, batch, batch_ids):
        input, labels = batch
        target_mask = (input["input_ids"] != 0)
        out = self(input, labels)
        loss = self.loss(out.float(), labels.float())
        out = torch.masked_select(out, target_mask)
        out = torch.sigmoid(out)
        labels = torch.masked_select(labels, target_mask)
        self.log("train_loss", loss, on_step=True, prog_bar=True)
        return {"loss": loss, "outs": out, "labels": labels}

    def training_epoch_end(self, training_ouput):
        out = np.concatenate([i["outs"].cpu().detach().numpy()
                              for i in training_ouput]).reshape(-1)
        labels = np.concatenate([i["labels"].cpu().detach().numpy()
                                 for i in training_ouput]).reshape(-1)
        auc = roc_auc_score(labels, out)
        self.print("train auc", auc)
        self.log("train_auc", auc)

    def validation_step(self, batch, batch_ids):
        input, labels = batch
        target_mask = (input["input_ids"] != 0)
        out = self(input, labels)
        loss = self.loss(out.float(), labels.float())
        out = torch.masked_select(out, target_mask)
        out = torch.sigmoid(out)
        labels = torch.masked_select(labels, target_mask)
        self.log("val_loss", loss, on_step=True, prog_bar=True)
        output = {"outs": out, "labels": labels}
        return {"val_loss": loss, "outs": out, "labels": labels}

    def validation_epoch_end(self, validation_ouput):
        out = np.concatenate([i["outs"].cpu().detach().numpy()
                              for i in validation_ouput]).reshape(-1)
        labels = np.concatenate([i["labels"].cpu().detach().numpy()
                                 for i in validation_ouput]).reshape(-1)
        auc = roc_auc_score(labels, out)
        self.print("val auc", auc)
        self.log("val_auc", auc)

In [ ]:
saint_plus = PlusSAINTModule()

# 학습

In [ ]:
trainer = pl.Trainer(gpus=-1, max_epochs=5, progress_bar_refresh_rate=21)

In [ ]:
trainer.fit(model=saint_plus,
            train_dataloader=train_loader,
            val_dataloaders=[val_loader, ])
